# Preprocessing

### 前処理
1. データ型の確認
1. データ形の変換(日付型)
1. データ形の変換(カテゴリ型の作成)
1. 基本統計量の算出
1. 単変量データの可視化
1. 異常値の置換・除去
1. 欠損値の補完・除去
1. 相関(数値×数値)の可視化：ペアプロット&相関行列
1. 相関(数値×カテゴリ)の可視化：ボックスプロット
1. 相関(カテゴリ×カテゴリ)の可視化：ヒートマップ

In [ ]:
# データ型の確認
data.info()

In [ ]:
# データ形の変換(日付型)
raw['Date'] = pd.to_datetime(raw['Date'])

In [ ]:
# データ形の変換(カテゴリ型の作成)
raw['Age_cat'] = pd.cut(raw['Age'], bins=np.arange(0, 100, 10), right=False)

In [ ]:
# データ形の変換(ID列をpd.DataframeのIndexへ変更)
raw = raw.set_index('PassengerId')
# raw = raw.set_index(raw['PassengerId'])#元の列名が残る方法

In [ ]:
# 基本統計量の算出
raw.describe(include='all')

#### 基本統計量の指標一覧
|指標|意味|
|---|---|
|count	|データの個数|
|unique	|ユニークな値の個数|
|top	|最頻値（最も多く出現したカテゴリ)|
|freq	|topで返されたカテゴリの出現回数|
|mean	|平均|
|std	|標準偏差|
|min	|最小値|
|25%	|第一四分位数|
|50%	|第二四分位数（中央値）|
|75%	|第三四分位数|
|max	|最大値|

### 単変量データの可視化
クロス集計をして各変数間の関係性をみるまえに、まずは各変数ごとに可視化をしてデータを理解します  
数値データはヒストグラム、カテゴリデータはカウントプロットで可視化すると分かりやすいです  
pandasのデフォルトのグラフでも可視化はできるますが、ヒストグラムは平均・中央値と比較してみたい、カウントプロットは実数も重ねてみたい、、ので関数を定義します

In [ ]:
# 数値データ用ヒストグラム
def num_vis(data):
    data.hist(figsize=(5, 4), color='darkblue', alpha=.7)
    mean = data.mean()
    median = data.median()
    ymax = pd.cut(data, 10).value_counts().max()
    plt.vlines(x=mean, ymin=0, ymax=ymax, colors='red', linestyles='--', lw=.7)# 平均値の直線追加
    plt.annotate('Mean: {}'.format(round(mean, 2)),xy=(mean, ymax), color='red')
    plt.vlines(x=median, ymin=0, ymax=ymax, colors='orange', linestyles='--', lw=.7)# 中央値の直線追加
    plt.annotate('Median: {}'.format(round(median, 2)),xy=(mean, ymax*0.8), color='orange')
    plt.title(data.name)
    plt.show()

In [ ]:
# カテゴリデータ用カウントプロット
def cut_vis(data):
    t = data.value_counts()
    t.plot.bar(figsize=(5, 4), color='indianred', alpha=.7)
    for i in range(len(t)):
        plt.annotate(t[i], xy=(i, t[i]/2), ha='center')#haでannotatioinのテキストを中央寄せ
    plt.title(data.name)   
    plt.show()

In [ ]:
# 関数を各変数へ適用
num_vis(raw['Age'])
num_vis(raw['Fare'])

cut_vis(raw['Label'])
cut_vis(raw['Sex'])
cut_vis(raw['Embarked'])

### 異常値の置換・除去

In [ ]:
#異常値として除去する対象を表示（確認）
raw[raw['Age'] > 100]

In [ ]:
#対象のIndexを取得して、pd.Dataframe.dropで除去
raw = raw.drop(raw[raw['Age'] > 100].index, axis=0)

In [ ]:
def replace_male(x):
    if pd.isna(x):
        pass
    else:
        return x.replace('man', 'male')

def replace_female(x):
    if pd.isna(x):
        pass
    else:
        return x.replace('woman', 'female')

raw['Sex'] = raw['Sex'].apply(replace_female)
raw['Sex'] = raw['Sex'].apply(replace_male)

### 欠損値の補完・除去

In [ ]:
# data = raw.dropna()
# 欠損値除去後のデータは別名にして、元データは残しておくと後々便利

In [ ]:
raw['Age'] = raw['Age'].fillna(raw['Age'].mean())
raw['Fare'] = raw['Fare'].fillna(raw['Fare'].mean())

### 相関(数値×数値)の可視化：ペアプロット&相関行列のヒートマップ

In [ ]:
sns.pairplot(data=raw, vars=['Age', 'Fare'], size=3)

ペアプロットは引数が色々あり、層別傾向や回帰直線も表現することができます  
kind='reg'で回帰直線を追加、diag_kind='kde'でヒストグラムをカーネル密度推定で表現、hueで層別塗り分けをします

In [ ]:
sns.pairplot(data=raw, vars=['Age', 'Fare'], hue='Sex', kind='reg', diag_kind='kde', size=3,
            plot_kws={'scatter_kws': {'alpha': 0.4},'line_kws': {'linestyle': '--', 'linewidth': .7}})

#### ヒートマップ
pd.Dataframe.corr()で相関行列が算出できるので、それをSeabornのヒートマップで可視化

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(raw[['Age', 'Fare']].corr(), fmt="1.2f", annot=True, lw=0.7, cmap='YlGnBu')

### 相関(数値×カテゴリ)の可視化：ボックスプロット
数値×カテゴリデータはボックスプロットでカテゴリごとの数値傾向を見るとデータの関係性が理解

In [ ]:
sns.boxplot(data=raw, x='Label', y='Age', palette="Set3")

全ての数値×カテゴリの組み合わせに適用

In [ ]:
plt.figure(figsize=(20, 7))
plt.subplots_adjust(wspace=0.1, hspace=0.4)
i = 1
for x in ['Label', 'Sex', 'Embarked', 'Age_cat']:
    for y in ['Age', 'Fare']:
        plt.subplot(3, 3,i)
        sns.boxplot(data=raw, x=x, y=y, palette="Set3")
        i += 1

### 相関(カテゴリ×カテゴリ)の可視化：ヒートマップ

カテゴリデータの関係性を把握するには、クロス集計表を出力して、ヒートマップで視覚的に把握するのが分かりやすいです  
pd.crosstabでカテゴリデータのクロス集計表を作成して、それをSeabornのヒートマップで可視化します

In [ ]:
sns.heatmap(pd.crosstab(raw['Label'], raw['Sex']), fmt="1.1f", annot=True, lw=0.7, cmap='Blues')

全てのカテゴリ×カテゴリの組み合わせに適用

In [ ]:
plt.figure(figsize=(20, 7))
plt.subplots_adjust(wspace=0.1, hspace=0.4)
i = 1
for label_1 in ['Label', 'Sex', 'Embarked']: 
    for label_2 in ['Label', 'Sex', 'Embarked']:
        plt.subplot(3, 3,i)
        sns.heatmap(pd.crosstab(raw[label_1], raw[label_2]), fmt="1.1f", annot=True, lw=0.7, cmap='Blues')

### pandas_profilingによるデータプロファイルレポートの自動作成

In [ ]:
import pandas_profiling as pdp
report = pdp.ProfileReport(raw)
report# notebook上にレポートを表示
# report.to_html# html形式でレポートを出力